In [68]:
library(tidyverse)

# Simulate Data

In [69]:
n <- 1000
set.seed(1000)
baseline <- data.frame(id=1:n, sex=rbinom(n,1, 0.5), age=rnorm(n, 25), visit = 0, drinking=rbinom(n, 1, 0.2))
head(baseline)

,id,sex,age,visit,drinking
,<int>,<int>,<dbl>,<dbl>,<int>
1,1,0,25.69171,0,0
2,2,1,25.38905,0,0
3,3,0,24.90537,0,0
4,4,1,23.05101,0,0
5,5,1,24.33441,0,0
6,6,0,25.01208,0,0


In [70]:
followup <-  data.frame(id=sample.int(n, 0.8*n),  visit = 1, drinking=rbinom(n*0.8, 1, 0.2))
head(followup)

,id,visit,drinking
,<int>,<dbl>,<int>
1,890,1,1
2,211,1,0
3,406,1,0
4,387,1,0
5,82,1,0
6,441,1,0


# Merge Data

In [71]:
followup_merge <-  followup %>% inner_join(baseline, by='id')
head(followup_merge)

,id,visit.x,drinking.x,sex,age,visit.y,drinking.y
,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>
1,890,1,1,0,24.00150,0,1
2,211,1,0,1,24.80642,0,0
3,406,1,0,1,25.02175,0,1
4,387,1,0,1,26.25194,0,0
5,82,1,0,0,26.98083,0,0
6,441,1,0,0,25.84911,0,0


In [72]:
followup_merge <-  followup %>% inner_join(baseline %>% select(-visit, -drinking), by='id') %>%
                    select(id, sex, age, everything())
head(followup_merge)

,id,sex,age,visit,drinking
,<int>,<int>,<dbl>,<dbl>,<int>
1,890,0,24.00150,1,1
2,211,1,24.80642,1,0
3,406,1,25.02175,1,0
4,387,1,26.25194,1,0
5,82,0,26.98083,1,0
6,441,0,25.84911,1,0


In [73]:
long_df <- rbind(baseline, followup_merge)
dim(long_df)
head(long_df)

[1] 1800    5

,id,sex,age,visit,drinking
,<int>,<int>,<dbl>,<dbl>,<int>
1,1,0,25.69171,0,0
2,2,1,25.38905,0,0
3,3,0,24.90537,0,0
4,4,1,23.05101,0,0
5,5,1,24.33441,0,0
6,6,0,25.01208,0,0


# Mutate

In [74]:
never_drinking <- long_df %>% group_by(id) %>% summarize(never_drinking = 1-prod(drinking))

In [75]:
long_df <- long_df %>% left_join(never_drinking, by = 'id')
head(long_df)

,id,sex,age,visit,drinking,never_drinking
,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>
1,1,0,25.69171,0,0,1
2,2,1,25.38905,0,0,1
3,3,0,24.90537,0,0,1
4,4,1,23.05101,0,0,1
5,5,1,24.33441,0,0,1
6,6,0,25.01208,0,0,1


# Factor

In [76]:
long_df <- long_df %>%  mutate(never_drinking = as.factor(as.character(never_drinking)), 
                    neverdrinker = fct_recode(never_drinking,   "Never drinker" = "1",  "Drinker" = "0" ) )
head(long_df)

,id,sex,age,visit,drinking,never_drinking,neverdrinker
,<int>,<int>,<dbl>,<dbl>,<int>,<fct>,<fct>
1,1,0,25.69171,0,0,1,Never drinker
2,2,1,25.38905,0,0,1,Never drinker
3,3,0,24.90537,0,0,1,Never drinker
4,4,1,23.05101,0,0,1,Never drinker
5,5,1,24.33441,0,0,1,Never drinker
6,6,0,25.01208,0,0,1,Never drinker


# Table

In [77]:
library(qwraps2)

In [78]:
demo_tab <- list(Demographics = list(Age = ~qwraps2::mean_sd(age)), Baseline_drinking = list(Drinking_baseline = ~qwraps2::n_perc(drinking) 
    ))


demo_tab[['Drinker classification']] <- list( Never_drinker = ~qwraps2::n_perc(neverdrinker == "Never drinker"),    Drinker = ~qwraps2::n_perc(neverdrinker == "Drinker")  ) 


In [79]:
table_df = long_df %>% filter(visit == 0) %>% drop_na()

options(qwraps2_markup = "markdown")
render_tab <- cbind( qwraps2::summary_table(table_df, 
    demo_tab),   qwraps2::summary_table(table_df %>% group_by(sex), demo_tab)  )

In [80]:
render_tab

,"table_df (N = 1,000)",sex: 0 (N = 488),sex: 1 (N = 512)
Age,24.99 &plusmn; 1.00,25.02 &plusmn; 0.97,24.97 &plusmn; 1.02
Drinking_baseline,186 (18.60%),90 (18.44%),96 (18.75%)
Never_drinker,931 (93.10%),455 (93.24%),476 (92.97%)
Drinker,69 (6.90%),33 (6.76%),36 (7.03%)


In [81]:
table_df = long_df %>% filter(visit == 0) %>% drop_na() %>% 
mutate(sex = as.factor(as.character(sex)),   sex = fct_recode(sex, "Female" = "1", "Male" = "0")   )

options(qwraps2_markup = "markdown")
render_tab <- cbind( qwraps2::summary_table(table_df, 
    demo_tab),   qwraps2::summary_table(table_df %>% group_by(sex), demo_tab)  )

In [82]:
render_tab

,"table_df (N = 1,000)",sex: Male (N = 488),sex: Female (N = 512)
Age,24.99 &plusmn; 1.00,25.02 &plusmn; 0.97,24.97 &plusmn; 1.02
Drinking_baseline,186 (18.60%),90 (18.44%),96 (18.75%)
Never_drinker,931 (93.10%),455 (93.24%),476 (92.97%)
Drinker,69 (6.90%),33 (6.76%),36 (7.03%)


In [83]:
qwraps2docx <- function(fname, render_tab) { 
    ## new way
    capture.output( print(render_tab), file = paste0(fname, '.md'))
    system( paste0("pandoc ", fname, ".md",  "  -o  ", fname, ".docx") )
}


qwraps2docx("stat_comp_test", render_tab)
